<a href="https://colab.research.google.com/github/lucaslzl/search_ia_p1/blob/master/Final_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introduction to Artificial Intelligence - MO416A**
**UNIVERSITY OF CAMPINAS**



This work was completed by the following members:



*   Aissa Hadj - 265189
*   Lucas Zanco Ladeira - 188951
*   Matheus Ferraroni - 212142
*   Maria Vitória Rodrigues Oliveira - 262884
*   Oscar Ciceri - 164786

The original code of the project is located on a [repository inside Github](https://github.com/lucaslzl/ga_ia_p2) and the video showing the search strategies working is on [youtube](https://youtube.com). 



# 1 - Introduction

The problem that will be tackled in this project is Feature Selection. It comprehends to obtain a subset of the features available in the dataset to improve the model results. By having irrelevant features in the dataset more processing and memory requirements are necessary. To summarize, some pros may be mentioned:
- Reduces Overfitting
- Improves Accurracy
- Reduces Training Time

Feature Selection may be done manually or automatically. Some manual techniques comprehend: univariate selection, feature importance, correlation matrix. Univariate selection has the goal to statistically describe the relation between each feature and the target. Feature importance generates a score for each feature to rank them, for instance, Decision Tree algorithms may rank features according to Gini impurity tests. Finally, Correlation Matrix show the correlation between each pair of features so that features with a high correlation may be removed. The literature presents the usage of optimization techniques to automatically find the best, or a quite good, subset of features. Some of the methods available comprehend:
- <b>Exaustive search</b>
- <b>Simulated Annealing</b>
- <b>Transformation Graph</b>
- <b>Genetic Algorithms</b>

<b>Exaustive Search</b> is not a optimization technique, but it is worth to be mentioned as the computational complexity $O(2^n)$. This technique tries every subset to find the best one. That presents how it is not feasible in most of the cases to use this technique. <b>Simulated Annealing</b> comprehends is a metaheuristic for complex nonlinear optimization problems and is based on the analogy of the simulation of the annealing of solids. The analogy pairs are as follows: feasible solution (state), cost (energy), optimal solution (ground state), local search (rapid quenching), simulated annealing (careful annealing). <b>Transformation Graph</b>. <b>Genetic Algorithms</b>.

# 2 - Genetic Algorithm

In [ ]:
import numpy as np
import threading

In [ ]:
class element:

    def __init__(self, idd, geracao, genome):
        self.idd = idd
        self.geracao = geracao
        self.genome = genome
        self.score = None


    def __repr__(self):
        return "(id="+str(self.idd)+",geracao="+str(self.geracao)+",score="+str(self.score)+")"

In [ ]:
def create_initial_population(self):
    for _ in range(self.population_size):
        self.population.append(element(self.elements_created, 0, self.random_genome()))
        self.elements_created += 1


def random_genome():
    return np.random.randint(low=0,high=2,size=len(df.columns),dtype=int)

In [ ]:
def run(self):

    while self.check_stop():
        self.calculate_score() 
        self.population.sort(key=lambda x: x.score, reverse=True) 

        if self.best_element_total==None or self.population[0].score > self.best_element_total.score: 
            self.best_element_total = self.population[0]

        self.do_log()

        if self.cut_half_population: 
            self.population = self.population[0:len(self.population)//2] 

        self.new_population()

        self.iteration_counter +=1

    return self.best_element_total

In [ ]:
def new_population(self):

    probs = self.get_probs()
    newPop = []
    best_replicator = int(self.population_size*self.replicate_best)

    while len(newPop)<self.population_size-best_replicator:
        parents = np.random.choice(self.population,size=2,p=probs) 

        if parents[0].score<parents[1].score: 
            parents = parents[::-1] 

        new_element = element(self.elements_created, self.iteration_counter, self.crossover(parents[0].genome, parents[1].genome))

        new_element.genome = self.active_mutate(new_element.genome)
        newPop.append(new_element)
        self.elements_created += 1

    for i in range(best_replicator):
        newPop.append(self.population[i])

    self.population = newPop

In [ ]:
def get_probs(self):
    if self.probs_type == 0:
        return self.probs_roulette()
    elif self.probs_type == 1:
        return self.probs_equal()


def probs_equal(self):
    return [1/len(self.population)]*len(self.population)


def probs_roulette(self):
    probs = [0]*len(self.population) 
    for i in range(len(probs)):
        probs[i] = self.population[i].score
    div = sum(probs)

    if div!=0:
        for i in range(len(probs)):
            probs[i] /= div
    else: 
        probs = self.probs_equal()
    return probs

In [ ]:
def check_stop(self):
    if self.stop_criteria_type==0:
        return self.stop_criteria_double()
    elif self.stop_criteria_type==1:
        return self.stop_criteria_iteration()
    elif self.stop_criteria_type==2:
        return self.stop_criteria_score()

def stop_criteria_double(self):
    s = self.population[0].score
    if s==None:
        s = 0
    return self.iteration_counter<self.iteration_limit or s>=self.max_possible_score

def stop_criteria_iteration(self):
    return self.iteration_counter<self.iteration_limit

def stop_criteria_score(self):
    s = self.population[0].score
    if s==None:
        s = 0
    return s>=self.max_possible_score

In [ ]:
def crossover(self, genA, genB):
    if self.crossover_type==0:
        return self.crossover_uniform(genA, genB)
    elif self.crossover_type==1:
        return self.crossover_single_point(genA, genB)
    elif self.crossover_type==2:
        return self.crossover_two_point(genA, genB)
    elif self.crossover_type==3:
        return self.crossover_rate_selection(genA, genB)

def crossover_rate_selection(self, genA, genB):
    new = np.array([],dtype=int)
    for i in range(len(genA)):
        if np.random.random()<self.crossover_rate:
            new = np.append(new, genA[i])
        else:
            new = np.append(new, genB[i])
    return new


def crossover_uniform(self, genA, genB):
    new = np.array([],dtype=int)
    for i in range(len(genA)):
        if np.random.random()<0.5:
            new = np.append(new, genA[i])
        else:
            new = np.append(new, genB[i])
    return new


def crossover_single_point(self, genA, genB):
    p = np.random.randint(low=1,high=len(genA)-1) 
    return np.append(genA[0:p],genB[p:])


def crossover_two_point(self, genA, genB):
    c1 = c2 = np.random.randint(low=0,high=len(genA)) 
    while c2==c1: 
        c2 = np.random.randint(low=0,high=len(genA))

    if c1>c2: 
        c1, c2 = c2,c1

    new = np.append(np.append(genA[0:c1],genB[c1:c2]),genA[c2:]) 
    return new

In [ ]:
def calculate_score(self):
    if self.use_threads: 

        threads_running = []
        for e in self.population:
            x = threading.Thread(target=self.thread_evaluate, args=(e,))
            x.start()
            threads_running.append(x)

        for i in range(len(threads_running)):
            threads_running[i].join()

    else: 
        for e in self.population:
            e.score = self.evaluate(e.genome)

def thread_evaluate(self, e):
    e.score = self.evaluate(e.genome)
    

def evaluate(genome):
    bool_genome = list(map(bool, genome))
    return model.evaluate(df.loc[:, bool_genome].copy(), target)

In [ ]:
def active_mutate(self,gen):
    if self.mutation_rate<=0: 
        return gen
    for i in range(len(gen)): 
        if np.random.random()<self.mutation_rate: 
            gen = self.mutate1(i, gen) 
    return gen


def mutate1(index, genome):
    if genome[index]==0:
        genome[index] = 1
    else:
        genome[index] = 0
    return genome


def mutate2(index, genome):
    aux = []
    for i in range(len(genome)):
        if i <= index:
            aux.append(genome[i])
        else:
            aux.insert(0, genome[i])
    genome = aux
    if genome[index]==0:
        genome[index] = 1
    else:
        genome[index] = 0
    return genome

# 3 - Methodology

# 4 - Results

# 5 - Conclusions